In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dress = pd.read_csv('/content/drive/MyDrive/Stylumia/Dataset/dress_clean.csv')
jeans = pd.read_csv('/content/drive/MyDrive/Stylumia/Dataset/jeans_clean.csv')

df = pd.concat([dress, jeans])

In [ ]:
dress.shape

(14609, 12)

In [ ]:
jeans.shape

(2874, 12)

In [ ]:
df.shape

(17483, 12)

In [ ]:
df1 = df.iloc[:2000]
df2 = df.iloc[2000:4000]
df3 = df.iloc[4000:6000]
df4 = df.iloc[6000:8000]
df5 = df.iloc[8000:10000]
df6 = df.iloc[10000:12000]
df7 = df.iloc[12000:14000]
df8 = df.iloc[14000:16000]
df9 = df.iloc[16000:17483]

In [ ]:
!pip install inference-sdk pandas requests pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [ ]:
import requests
from PIL import Image
import io
import time
from tqdm import tqdm
from inference_sdk import InferenceHTTPClient

In [ ]:
CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="2gzq3x8MFv9nEFXCzuEe"
)

MODEL_ID = "product-fashion-matching-02/2"

In [ ]:
def get_bounding_boxes(image_url):
    try:
        # Download the image
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(io.BytesIO(response.content))

        # Save temporarily
        temp_path = "temp_image.jpg"
        image.save(temp_path)

        # Inference
        result = CLIENT.infer(temp_path, model_id=MODEL_ID)

        boxes = []
        for pred in result['predictions']:
            x, y = pred['x'], pred['y']
            w, h = pred['width'], pred['height']
            cls = pred['class']
            conf = pred['confidence']

            left = x - w / 2
            top = y - h / 2
            right = x + w / 2
            bottom = y + h / 2

            boxes.append({
                "class": cls,
                "confidence": conf,
                "bbox": [left, top, right, bottom]
            })

        return boxes
    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return None

In [ ]:
def apply_with_progress(df, column_name, func):
    results = []
    total = len(df)
    start_time = time.time()

    for i, val in enumerate(tqdm(df[column_name], total=total)):
        result = func(val)
        results.append(result)

        elapsed = time.time() - start_time
        avg_time = elapsed / (i + 1)
        remaining = avg_time * (total - i - 1)
        tqdm.write(f"Processed {i+1}/{total}, Estimated time remaining: {remaining:.1f} seconds")

    return results

In [ ]:
df1['box'] = apply_with_progress(df1, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:02<1:14:16,  2.23s/it]

Processed 1/2000, Estimated time remaining: 4465.6 seconds


  0%|          | 2/2000 [00:04<1:22:10,  2.47s/it]

Processed 2/2000, Estimated time remaining: 4864.0 seconds


  0%|          | 3/2000 [00:05<56:01,  1.68s/it]  

...

100%|█████████▉| 1999/2000 [24:19<00:00,  1.51it/s]

Processed 1999/2000, Estimated time remaining: 0.7 seconds


100%|██████████| 2000/2000 [24:19<00:00,  1.37it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df1.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df1.csv', index=False)

In [ ]:
df2['box'] = apply_with_progress(df2, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<28:00,  1.19it/s]

Processed 1/2000, Estimated time remaining: 1678.4 seconds


  0%|          | 2/2000 [00:01<31:55,  1.04it/s]

Processed 2/2000, Estimated time remaining: 1878.1 seconds


  0%|          | 3/2000 [00:02<26:28,  1.26it/s]

...

100%|█████████▉| 1999/2000 [31:04<00:00,  1.55it/s]

Processed 1999/2000, Estimated time remaining: 0.9 seconds


100%|██████████| 2000/2000 [31:05<00:00,  1.07it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df2.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df2.csv', index=False)

In [ ]:
df3['box'] = apply_with_progress(df3, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<26:30,  1.26it/s]

Processed 1/2000, Estimated time remaining: 1598.8 seconds


  0%|          | 2/2000 [00:01<25:03,  1.33it/s]

Processed 2/2000, Estimated time remaining: 1520.7 seconds


  0%|          | 3/2000 [00:02<25:59,  1.28it/s]

...

100%|█████████▉| 1999/2000 [26:44<00:00,  1.43it/s]

Processed 1999/2000, Estimated time remaining: 0.8 seconds


100%|██████████| 2000/2000 [26:45<00:00,  1.25it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df3.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df3.csv', index=False)

In [ ]:
df4['box'] = apply_with_progress(df4, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<24:11,  1.38it/s]

Processed 1/2000, Estimated time remaining: 1437.7 seconds


  0%|          | 2/2000 [00:01<23:43,  1.40it/s]

Processed 2/2000, Estimated time remaining: 1427.0 seconds


  0%|          | 3/2000 [00:02<21:50,  1.52it/s]

...

100%|█████████▉| 1999/2000 [33:08<00:00,  1.34it/s]

Processed 1999/2000, Estimated time remaining: 1.0 seconds


100%|██████████| 2000/2000 [33:09<00:00,  1.01it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df4.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df4.csv', index=False)

In [ ]:
df5['box'] = apply_with_progress(df5, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<21:21,  1.56it/s]

Processed 1/2000, Estimated time remaining: 1279.6 seconds


  0%|          | 2/2000 [00:01<19:28,  1.71it/s]

Processed 2/2000, Estimated time remaining: 1184.9 seconds


  0%|          | 3/2000 [00:01<19:54,  1.67it/s]

...

100%|█████████▉| 1999/2000 [24:57<00:00,  1.06it/s]

Processed 1999/2000, Estimated time remaining: 0.7 seconds


100%|██████████| 2000/2000 [24:58<00:00,  1.33it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df5.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df5.csv', index=False)

In [ ]:
df6['box'] = apply_with_progress(df6, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<19:39,  1.69it/s]

Processed 1/2000, Estimated time remaining: 1179.0 seconds


  0%|          | 2/2000 [00:01<20:35,  1.62it/s]

Processed 2/2000, Estimated time remaining: 1226.1 seconds


  0%|          | 3/2000 [00:01<22:37,  1.47it/s]

...

100%|█████████▉| 1999/2000 [25:53<00:00,  1.29it/s]

Processed 1999/2000, Estimated time remaining: 0.8 seconds


100%|██████████| 2000/2000 [25:54<00:00,  1.29it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df6.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df6.csv', index=False)

In [ ]:
df7['box'] = apply_with_progress(df7, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<23:00,  1.45it/s]

Processed 1/2000, Estimated time remaining: 1383.1 seconds


  0%|          | 2/2000 [00:01<23:50,  1.40it/s]

Processed 2/2000, Estimated time remaining: 1424.2 seconds


  0%|          | 3/2000 [00:02<23:37,  1.41it/s]

...

100%|█████████▉| 1999/2000 [27:00<00:01,  1.31s/it]

Processed 1999/2000, Estimated time remaining: 0.8 seconds


100%|██████████| 2000/2000 [27:01<00:00,  1.23it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df7.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df7.csv', index=False)

In [ ]:
df8['box'] = apply_with_progress(df8, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/2000 [00:00<22:18,  1.49it/s]

Processed 1/2000, Estimated time remaining: 1338.2 seconds


  0%|          | 2/2000 [00:01<22:02,  1.51it/s]

Processed 2/2000, Estimated time remaining: 1324.2 seconds


  0%|          | 3/2000 [00:01<22:07,  1.50it/s]

...

100%|█████████▉| 1999/2000 [29:33<00:01,  1.45s/it]

Processed 1999/2000, Estimated time remaining: 0.9 seconds


100%|██████████| 2000/2000 [29:34<00:00,  1.13it/s]

Processed 2000/2000, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df8.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df8.csv', index=False)

In [ ]:
df9['box'] = apply_with_progress(df9, 'feature_image_s3', get_bounding_boxes)

  0%|          | 1/1483 [00:00<18:37,  1.33it/s]

Processed 1/1483, Estimated time remaining: 1116.2 seconds


  0%|          | 2/1483 [00:01<15:51,  1.56it/s]

Processed 2/1483, Estimated time remaining: 975.6 seconds


  0%|          | 3/1483 [00:02<17:50,  1.38it/s]

...

100%|█████████▉| 1482/1483 [24:19<00:00,  1.30it/s]

Processed 1482/1483, Estimated time remaining: 1.0 seconds


100%|██████████| 1483/1483 [24:19<00:00,  1.02it/s]

Processed 1483/1483, Estimated time remaining: 0.0 seconds



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df9.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df9.csv', index=False)

In [ ]:
df_combined = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)

In [ ]:
df_combined.to_csv('/content/drive/MyDrive/Stylumia/Dataset/df_combined.csv', index=False)

In [ ]:
df_combined.head()

,selling_price,discount,product_id,sku,brand,mrp,product_name,feature_image_s3,channel_id,feature_list,description,style_attributes,box
0,474848.9539,0.0,b613d7b5dfe86f3e695d931d31fd729fdf44e181f14079...,479495,RALPH LAUREN,474848.9539,Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{},NaN
1,464648.6919,0.0,482b10a23f8d00cfc7c9bbeeac4e26d25dd303d8e62e97...,502670,RALPH LAUREN,464648.6919,Kristian Silk Tuxedo Dress,https://gallery.stylumia.com/originals/2020/02...,14,"['Slim fit. Designed to hit at the ankle.', 'U...",The Kristian evening dress is informed by the ...,{},NaN
2,29496.0812,0.0,3508b052ef7a5eea820423b97713612bc92a3f2301a3d3...,478766,RALPH LAUREN,29496.0812,Fit-and-Flare Shirtdress,https://gallery.stylumia.com/originals/2020/02...,14,['Fit-and-flare silhouette. Intended to hit at...,Airy georgette and a flattering fit-and-flare ...,{},NaN
3,17156.9392,0.0,6360245240b68885bd4dbcef8d8856c0fb13f1314769f5...,50297209_041,BHLDN,17156.9392,Adena Crepe Dress,https://gallery.stylumia.com/originals/2020/05...,48,"['Back zip', 'Polyester; polyester lining', 'P...","A sleek square neckline tops this stretchy, bo...","{'modelNotes': '', 'dimensions': 'Fits true to...",NaN
4,26079.5467,0.0,5d07037957e64d1e218499cb7d7a8e5e57aa59249bb806...,478750,RALPH LAUREN,26079.5467,Belted Cotton-Blend Shirtdress,https://gallery.stylumia.com/originals/2021/02...,14,['Fit-and-flare silhouette Designed to hit at ...,This iteration of Lauren's iconic shirtdress i...,{},NaN
